In [3]:
# script to convert *.las files to potree 1.6 compatible octrees using laspy and numpy.
# A great source for *laz files is http://3dsm.bk.tudelft.nl/matahn 
#
# Code is setup to enable parallel processing
# TODO: optimize code
# TODO: allow multicore processing
# TODO: allow more paramaters than just x,y,z to be parsed
# TODO: change code to module
# TODO: 

In [4]:
import os
import laspy 
import numpy as np
from pprint import pprint
import json
from pathlib import Path
import copy
from contexttimer import Timer
import pandas as pd

In [5]:
# convert laz to las using las2las
LAS_DIRECTORY = 'E:\pointclouds'
p = Path(LAS_DIRECTORY)
for f in p.glob('*.laz'):
    fn = f.__str__()
    fn_new = fn.replace('.laz', '.las')
    if not os.path.exists(fn_new):
        cmd = r'"C:\Program Files (x86)\lastools\bin\las2las.exe" -i {} -o {} '.format(fn,fn_new)
        os.system(cmd)

In [6]:
# make a generator for file headers
# http://pythonhosted.org/laspy/header.html
p = Path(LAS_DIRECTORY)
def headers():
    for f in p.glob('*.las'):
        fn = f.__str__()
        lasfile = laspy.file.File(fn)
        yield lasfile.header    

In [7]:
bounds = np.array((
    np.max([hdr.max for hdr in headers()],axis=0),
    np.min([hdr.min for hdr in headers()],axis=0),
    ))
print(bounds)
[print(hdr.scale) for hdr in headers()]

[[  9.25141500e+04   4.38354630e+05   1.61280000e+02]
 [  9.13902400e+04   4.37388640e+05  -1.98100000e+01]]
[0.01, 0.01, 0.01]


[None]

In [8]:
# define maximum bounding cube for data. MUST be cube, so all sides equal length
min_size = max(bounds[0,:] - bounds[1,:])
print(min_size)
suggested_centers = ((bounds[0,:] + bounds[1,:])/2).astype(np.dtype('<i4'))
suggested_levels = np.ceil(np.log2(min_size/100))
print(suggested_levels,suggested_centers)

1123.91
4.0 [ 91952 437871     70]


In [45]:
CENTERS = {'x': 125860, 'y':402844, 'z':-50}
SMALLEST_TILE_SIZE = 25 # meter
LEVELS = 12
SCALE = 0.01
OUTPUT_DIR = Path(r'D:\pointclouds\potree\data')
# define size as nice power of two (not necessary)
SIZE = SMALLEST_TILE_SIZE * 2 ** LEVELS
BBOX = {k: np.array([v-SIZE/2, v+SIZE/2]).astype('i4') for k, v in CENTERS.items()}

print(BBOX)
# check if all points are within bounding box
print([[h_min>=BBOX[dim][0] and h_max<=BBOX[dim][1] for
       dim, h_min, h_max in zip(['x','y','z'],
                                hdr.min,
                                hdr.max)] for
       hdr in headers()])



{'y': array([351644, 454044]), 'x': array([ 74660, 177060]), 'z': array([-51250,  51150])}
[]


In [46]:
HIERACHY_STEP_SIZE = 5
def address_to_path(address):
    if address is '':
        return Path(OUTPUT_DIR,'r','r.bin')

    parts = [address[i:i+HIERACHY_STEP_SIZE] for i in range(0, len(address)+1, HIERACHY_STEP_SIZE)]
    parts[-1]='r' + address + '.bin'
    return Path(OUTPUT_DIR,'r',*parts)

print(address_to_path('000'))
print(address_to_path('112312312'))  
print(address_to_path('7000'))
print(address_to_path(''))


D:\pointclouds\potree\data\r\r000.bin
D:\pointclouds\potree\data\r\11231\r112312312.bin
D:\pointclouds\potree\data\r\r7000.bin
D:\pointclouds\potree\data\r\r.bin


In [47]:
def address_2_origin(address):
    """ Each tile is saved in coordinates relitive to the origin of the tile.
    The origin of a file can be calculated from the bounding box (BBOX) and the address
    """
    if address == '':
        address = '0'

    # return origin relative to bounding box origin in unscaled values
    level = len(address)
    bin_parts = [bin(int(c)+8)[3::] for c in address]
    x0 = int((int(''.join([b[0] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    y0 = int((int(''.join([b[1] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    z0 = int((int(''.join([b[2] for b in bin_parts]),2) / 2**level) * SIZE / SCALE)
    return (x0, y0, z0)

print(*address_2_origin('000'))
print(*address_2_origin('112312312'))  
print(*address_2_origin('7000'))
print(*address_2_origin(''))

0 0 0
0 2180000 8760000
5120000 5120000 5120000
0 0 0


In [48]:
p = Path(OUTPUT_DIR,'r')
for n in range(0,LEVELS):
    print('removed',len([f.unlink() for f in p.rglob('r'+ ('?' * n) + '.bin')]),'files at level', n)

removed 0 files at level 0
removed 0 files at level 1
removed 0 files at level 2
removed 0 files at level 3
removed 0 files at level 4
removed 0 files at level 5
removed 0 files at level 6
removed 0 files at level 7
removed 0 files at level 8
removed 0 files at level 9
removed 0 files at level 10
removed 0 files at level 11


In [49]:
def process_chunk(data,bins,hdr):
    # put all points in bins
    # Simple was: 
    #    binned = np.vstack((np.digitize(d, b) for d, b in zip(data,bins)))
    # optimiziation: 
    #    remove bins smaller than min in data before digitize, is huge speed improvement
    binned = np.vstack(
        (np.digitize(d, np.extract(b>=d.min(),b)) + np.where(b>=d.min())[0][0]
         for d, b in zip(data,bins)))

    for x in np.unique(binned[0]):
        ix = np.equal(binned[0],x)
        data_x = data[:,ix]
        binned_x = binned[:,ix]
        address_x = bin(x + 2**LEVELS)[-LEVELS::]

        for y in np.unique(binned_x[1]):
            iy = np.equal(binned_x[1],y)
            data_xy = data_x[:,iy]
            binned_xy = binned_x[:,iy]
            address_y = bin(y + 2**LEVELS)[-LEVELS::]
            
            for z in np.unique(binned_xy[2]):
                iz = np.equal(binned_xy[2],z)
                data_xyz = data_xy[:,iz]
                address_z = bin(z + 2**LEVELS)[-LEVELS::]
                
                # make filename of 0-7's
                # https://github.com/potree/potree/blob/master/docs/file_format.md
                address = ''.join([str(int(''.join((x,y,z)),2)) for x,y,z in zip(address_x,address_y,address_z)])
                
                filename = address_to_path(address).__str__()

                # convert points from int32 relative to hdr.scale and hdr.offset to uint32 relative to SCALE, BBOX
                # and the tile box address
                x0, y0, z0 = address_2_origin(address)

                if not os.path.exists(os.path.dirname(filename)):
                    os.makedirs(os.path.dirname(filename))
                with open(filename,'a') as f:
                    np.vstack((
                         (((data_xyz[0] * hdr.scale[0]) + hdr.offset[0] - BBOX['x'][0]) / SCALE).astype('<u4') - x0,
                         (((data_xyz[1] * hdr.scale[1]) + hdr.offset[1] - BBOX['y'][0]) / SCALE).astype('<u4') - y0,
                         (((data_xyz[2] * hdr.scale[2]) + hdr.offset[2] - BBOX['z'][0]) / SCALE).astype('<u4') - z0
                         )).transpose().tofile(f)

In [50]:
def process_file(fn, t, processed):

    with laspy.file.File(fn) as lasfile:
        hdr = lasfile.header
        # check if all points are within bounding box
        print([h_min>=BBOX[dim][0] and h_max<=BBOX[dim][1] for
               dim, h_min, h_max in zip(['x','y','z'],
                                        hdr.min,
                                        hdr.max)])

        # to avoid duplicating the point coordinates in memory, discretize the points based on their raw coordinates
        # in unscaled integers
        bbox_unscaled = {dim: ((BBOX[dim]-offset)/scale).astype('i4') for 
                       dim, scale, offset in zip(['x','y','z'], hdr.scale, hdr.offset)}


        # determine the bins to classify points
        bins = tuple(np.linspace(bbox_unscaled[dim][0],
                            bbox_unscaled[dim][1],
                            2**(LEVELS)+1
                           ).astype('i4')[1:-1] for dim in ['x','y','z'])

        for n in range(0,hdr.count,PROCESS_CHUNK_SIZE):
            with laspy.file.File(fn) as lasfile2:
                data = np.vstack((lasfile2.X[n:n+PROCESS_CHUNK_SIZE],
                                  lasfile2.Y[n:n+PROCESS_CHUNK_SIZE],
                                  lasfile2.Z[n:n+PROCESS_CHUNK_SIZE]))
                process_chunk(data,bins,hdr)
                processed += lasfile2.X[n:n+PROCESS_CHUNK_SIZE].size
            print('processed',processed,'points in ',t.elapsed)
    return processed

In [51]:
PROCESS_CHUNK_SIZE = int(1e7)
with Timer() as t:
    p = Path(LAS_DIRECTORY)
    processed = 0
    for fn in p.glob('*.las'):
        processed += process_file(fn.__str__(),t,processed)
     


[True, True, True]
processed 10000000 points in  16.443815290871328
processed 16864177 points in  31.81358622355458


In [52]:
import re
only_digits = re.compile(r"\D")

p = Path(OUTPUT_DIR,'r')
addresses = [only_digits.sub("",paths.parts[-1]) for paths in p.glob('**/r*.bin')]
print(addresses[::100])

['324242244607', '324242264656', '324242600663', '324242602472', '324242606025', '324242606476', '324242620412', '324242622252', '324242624076', '324242626061', '324242626661', '324242642056', '324242642527', '324242644661', '324242646403', '324242660017', '324242660576', '324242662434', '324242664146', '324242664601', '324242666256', '324246200276', '324246202261', '324246204234', '324246220102', '324246220636', '324246222414', '324246224210', '324260044454', '324260400205', '324260402001', '324260404041', '324260404647', '324260440014', '324260440616', '324260442472', '324260444455', '324260446423', '324264000245', '324264002012', '324264004043']


In [53]:
# convert tree to tree
tree = {}
for a in addresses:
    t = tree
    for s in a:
        if s not in t:
            t[s] = {} 
        t = t[s]

In [54]:
# print(json.dumps(tree,sort_keys=True, indent=2, separators=(',', ': ')))
# convert to list of all possible tiles, and their children

def nodes_to_list(prefix, key, value):
    hrc = [{'address':prefix+key,
            'children': tuple(value.keys())}, 
          ]
    
    [hrc.extend(nodes_to_list(prefix+key,k,v)) for k, v in value.items()]
    
    return hrc

hrc = nodes_to_list('','',tree)
hrc.sort(key=lambda k: (len(k['address']), k['address']))


In [55]:
for item in hrc:
    path = address_to_path(item['address'])
    item.update(
        {
            'mask': sum([2 ** int(c) for c in item['children']]),
            'path': path,
            'parentPath': address_to_path(item['address'][:-1])
        })
    
pprint(hrc[0])
pprint(hrc[-1])

{'address': '',
 'children': ('3',),
 'mask': 8,
 'parentPath': WindowsPath('D:/pointclouds/potree/data/r/r.bin'),
 'path': WindowsPath('D:/pointclouds/potree/data/r/r.bin')}
{'address': '324264006251',
 'children': (),
 'mask': 0,
 'parentPath': WindowsPath('D:/pointclouds/potree/data/r/32426/40062/r32426400625.bin'),
 'path': WindowsPath('D:/pointclouds/potree/data/r/32426/40062/r324264006251.bin')}


In [56]:
dt = np.dtype([('x','<u4'),('y','<u4'),('z','<u4')])
def read_file_at_address(address):
    with address_to_path(address).open('r') as f:
        points = np.fromfile(f,dtype=dt)
    x0, y0, z0 = address_2_origin(address)
    points['x'] += x0
    points['y'] += y0
    points['z'] += z0
    return(points)

In [57]:
def thin_points(points,level):
    """ thin the data in a file
    bin the data in 3d cubes, and keep only one point per level
    """
    
    df = pd.DataFrame(points)
    n_bins = 256
    size = SMALLEST_TILE_SIZE * 2 ** (LEVELS - level) / SCALE
    bins = np.linspace(0,size,n_bins+1)
    binned = copy.copy(points)
    for dim in ['x','y','z']:
        df['bin_'+dim] = np.digitize(points[dim],bins) 
    df.drop_duplicates(inplace=True,subset=('bin_x','bin_y','bin_z'))
    subset = df.as_matrix(columns=('x','y','z'))
    return(subset)

In [58]:
# loop over all files in tree
for h in hrc[::-1]:
    # if file doesn't exist, generate it from it's children
    if not h['path'].exists():
        points = np.hstack([read_file_at_address(h['address']+child) for child in h['children']])
        
        x0, y0, z0 = address_2_origin(h['address'])
        points['x'] -= x0
        points['y'] -= y0
        points['z'] -= z0
        subset = thin_points(points,len(h['address']))
        with h['path'].open('w') as f:
            subset.tofile(f)
        
    h['n_points'] = int(h['path'].stat().st_size / dt.itemsize)
        

In [59]:
hrc_dtype = np.dtype([
    ("mask", "<u1"),            
    ("n_points", "<u4"),   
    ])

import re
only_digits = re.compile(r"\D")
output_dir = Path(OUTPUT_DIR,'r').__str__()

for d in set(h['path'].parent for h in hrc):
    hrc_data = np.array(
        [(h['mask'],h['n_points']) for h in hrc if 
         h['parentPath'].parent == d or h['path'].parent == d],
        dtype=hrc_dtype)
    path = d.joinpath('r' + only_digits.sub("",d.__str__().replace(output_dir,'')) + '.hrc')
    with path.open('w') as f:
        hrc_data.tofile(f)

In [69]:
options = {
    "version": "1.6",
    "octreeDir": OUTPUT_DIR.parts[-1],
    "boundingBox": {
        "lx": int(BBOX['x'][0]),
        "ly": int(BBOX['y'][0]),
        "lz": int(BBOX['z'][0]),
        "ux": int(BBOX['x'][1]),
        "uy": int(BBOX['y'][1]),
        "uz": int(BBOX['z'][1])
    },
    "tightBoundingBox": {
        "lx": bounds[1,0],
        "ly": bounds[1,1],
        "lz": bounds[1,2],
        "ux": bounds[0,0],
        "uy": bounds[0,1],
        "uz": bounds[0,2]
    },
    "pointAttributes": ["POSITION_CARTESIAN"],
    "spacing": 0.01,
    "scale": SCALE,
    "hierarchyStepSize": HIERACHY_STEP_SIZE,
}
print(json.dumps(options,sort_keys=True, indent=2, separators=(',', ': ')))

with OUTPUT_DIR.parent.joinpath('cloud.js').open('w') as f:
    json.dump(options,f)

{
  "boundingBox": {
    "lx": 74660,
    "ly": 351644,
    "lz": -51250,
    "ux": 177060,
    "uy": 454044,
    "uz": 51150
  },
  "hierarchyStepSize": 5,
  "octreeDir": "data",
  "pointAttributes": [
    "POSITION_CARTESIAN"
  ],
  "scale": 0.01,
  "spacing": 0.01,
  "tightBoundingBox": {
    "lx": 91390.24,
    "ly": 437388.64,
    "lz": -19.81,
    "ux": 92514.15000000001,
    "uy": 438354.63,
    "uz": 161.28
  },
  "version": "1.6"
}


In [65]:
OUTPUT_DIR.parts[-1]

'data'